# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

#### Analyis
Observed Trends
1.  
2.  
3.  

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [16]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=100)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=100)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
# len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [17]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [22]:
import json

temp = []
humidities = []
cloudiness = []
wind_speeds = []
lat = []
lng = []
country = []
date = []

city_data = []

# response = requests.get(query_url + "&q=" + city).json()

for i, city in enumerate(cities):
    try:
        response = requests.get(query_url + city).json()

        temp.append(response["main"]['temp_max'])
        humidities.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speeds.append(response['wind']['speed'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        country.append(response['sys']['country'])
        date.append(response['dt'])

    except:
        pass
        
city_data.append({"City": cities,
            "Lat": lat,
            "Lng": lng,
            "Max Temp": temp,
            "Humidity": humidities,
            "Cloudiness": cloudiness,
            "Wind Speed": wind_speeds,
            "Country": country,
            "Date": date})




In [23]:


print(city_data[0])

{'City': ['kapaa', 'lwowek slaski', 'atuona', 'northam', 'tripoli', 'hithadhoo', 'bluff', 'georgetown', 'unidad', 'yumen', 'muros', 'busselton', 'labutta', 'mataura', 'belushya guba', 'albany', 'lianzhou', 'korla', 'mahebourg', 'cherskiy', 'luderitz', 'porto belo', 'marfino', 'zambezi', 'vaini', 'rincon', 'coquimbo', 'hun', 'deep river', 'sao francisco', 'reutte', 'bilibino', 'sentyabrskiy', 'taolanaro', 'andenes', 'yabelo', 'ushuaia', 'manbij', 'coulommiers', 'tidore', 'berlevag', 'kaitangata', 'yellowknife', 'rikitea', 'krasnoselkup', 'broome', 'jamestown', 'promyshlennyy', 'ushtobe', 'butaritari', 'louisbourg', 'longyearbyen', 'ancud', 'gorno-chuyskiy', 'avarua', 'bredasdorp', 'hilo', 'rocha', 'mount vernon', 'khanpur', 'esperance', 'katsuura', 'saint george', 'namibe', 'bowen', 'kruisfontein', 'macieira de cambra', 'faanui', 'glushkovo', 'egvekinot', 'qaanaaq', 'boyuibe', 'mahanoro', 'bengkulu', 'toliary', 'denpasar', 'tuktoyaktuk', 'east london', 'hailar'], 'Lat': [22.0752, 51.110

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [24]:
# weather_dict = {
#     "city": cities,
#     "lat": lat,
#     "lng": lng,
#     "max temp": temp,
#     "humidity": humidities,
#     "cloudiness": cloudiness,
#     "wind speed": wind_speeds,
#     "country": country, 
#     "date": date
# }

weather_df = pd.DataFrame(city_data[0])
weather_df.head()

# city_data[0:10]
# for city in city_data:
#     print(len(city))

ValueError: All arrays must be of the same length

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [7]:
#  Get the indices of cities that have humidity over 100%.


In [8]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs in the output_data folder
* Below are examples of what you should get but your results will be different.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression